In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from pathlib import Path
import datetime


path = Path.cwd().parent
raw_data = str(path) + '/raw_data/'
data = str(path) + '/data/'

# 데이터 불러오기

In [6]:
# 불러오는 데이터의 날짜(작업장비이력 테이블의 가장 최근 날짜)


FMSA10_작업지시_결과 = pd.read_csv(data+'FMSA10_작업지시_결과_ver2.csv')

FMSA11_작업장비이력 = pd.read_excel(raw_data + 'FMSA11_작업장비이력.xlsx')

FMSX05_장비마스터 = pd.read_csv(data + 'FMSX05_장비마스터_ver2.csv')

FMSB07_자재출고 = pd.read_excel(raw_data + 'FMSB07_자재출고.xlsx')

FMSB03_자재마스터 = pd.read_csv(data + 'FMSB03_자재마스터_ver2.csv')

FMSX01_건물정보 = pd.read_excel(raw_data+'FMSX01_건물정보.xlsx')

FMSB03_자재마스터.drop(['class_nm_4','class_cd_4'],axis =1,inplace = True)

FMSB03_자재마스터.rename(columns={'class_nm_3':'jajae_class_nm_3'},inplace = True)
FMSB03_자재마스터.rename(columns={'class_nm_2':'jajae_class_nm_2'},inplace = True)
FMSB03_자재마스터.rename(columns={'class_nm_1':'jajae_class_nm_1'},inplace = True)

FMSB03_자재마스터.rename(columns={'class_cd_3':'jajae_class_cd_3'},inplace = True)
FMSB03_자재마스터.rename(columns={'class_cd_2':'jajae_class_cd_2'},inplace = True)
FMSB03_자재마스터.rename(columns={'class_cd_1':'jajae_class_cd_1'},inplace = True)


FMSX05_장비마스터.rename(columns={'class_nm_4':'jangbi_class_nm_4'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_nm_3':'jangbi_class_nm_3'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_nm_2':'jangbi_class_nm_2'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_nm_1':'jangbi_class_nm_1'},inplace = True)

fmsx01 = FMSX01_건물정보[['bd_cd','bd_nm']]
FMSX05_장비마스터 = pd.merge(fmsx01,FMSX05_장비마스터, on='bd_cd', how = 'right')


FMSX05_장비마스터.rename(columns={'class_cd_4':'jangbi_class_cd_4'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_cd_3':'jangbi_class_cd_3'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_cd_2':'jangbi_class_cd_2'},inplace = True)
FMSX05_장비마스터.rename(columns={'class_cd_1':'jangbi_class_cd_1'},inplace = True)

FMSA10_작업지시_결과.rename(columns={'class_nm_4':'job_class_nm_4'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_nm_3':'job_class_nm_3'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_nm_2':'job_class_nm_2'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_nm_1':'job_class_nm_1'},inplace = True)


FMSA10_작업지시_결과.rename(columns={'class_cd_4':'job_class_cd_4'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_cd_3':'job_class_cd_3'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_cd_2':'job_class_cd_2'},inplace = True)
FMSA10_작업지시_결과.rename(columns={'class_cd_1':'job_class_cd_1'},inplace = True)

fmsa10 =FMSA10_작업지시_결과[['job_cd','job_nm','job_jisi_dt','job_enddt','job_shm','job_ehm','job_class_nm_1','job_class_nm_2','job_class_nm_3','job_class_nm_4']]
fmsb03 = FMSB03_자재마스터[['mat_cd','mat_nm','jajae_class_nm_1','jajae_class_nm_2','jajae_class_nm_3','jajae_class_cd_3']]
fmsx05 = FMSX05_장비마스터[['bd_cd','bd_nm','fl_cd','fac_cd','fac_nm','jangbi_class_nm_1','jangbi_class_nm_2','jangbi_class_nm_3','jangbi_class_nm_4','fac_instdt']]


FMSB07_자재출고["mat_cd"] = FMSB07_자재출고["mat_cd"].astype("str")
FMSB07_자재출고["mat_cd"] = list(map(lambda x: x[:-3],FMSB07_자재출고["mat_cd"].tolist())) # FMSB03과 inner join하기 위해 mat_cd 형식 일치
fmsb03["mat_cd"] = fmsb03["mat_cd"].astype("str")
jajae = pd.merge(FMSB07_자재출고,fmsb03 ,on = 'mat_cd', how = 'inner')
jajae = jajae[['job_cd','outw_day','mat_cd','outw_qty','mat_nm_x','jajae_class_nm_1','jajae_class_nm_2','jajae_class_nm_3','jajae_class_cd_3']]


jangbi = pd.merge(FMSA11_작업장비이력, fmsx05, how='inner', on='fac_cd')
jangbi_broken = pd.merge(fmsa10,jangbi, on='job_cd',how = 'inner')

jangbi_broken.drop(['job_cont','chg_sysdt','chg_emp_no'],axis=1,inplace=True)
jangbi_broken = jangbi_broken.dropna()

jangbi_broken['자재매칭여부'] = jangbi_broken['job_cd']
arr_장비 = list(jangbi_broken['job_cd'])
arr_자재 = list(jajae['job_cd'])
#자재 매칭이 안되는 경우
arr = list(set(arr_장비)- set(arr_자재))

def function(x):
    if x in arr:
        return 0
    else:
         return 1

jangbi_broken['자재매칭여부'] = jangbi_broken['자재매칭여부'].apply(function)
df = pd.merge(jangbi_broken,jajae, on='job_cd',how = 'inner')

final = df[['job_cd','job_nm','job_jisi_dt','job_enddt','job_shm','job_ehm','job_class_nm_1','job_class_nm_2','job_class_nm_3','job_class_nm_4','fac_cd','fac_nm','fac_instdt','mat_cd','mat_nm_x',
           'jangbi_class_nm_1','jangbi_class_nm_2','jangbi_class_nm_3','jangbi_class_nm_4','자재매칭여부','mat_nm_x','jajae_class_nm_1','jajae_class_nm_2','jajae_class_nm_3','jajae_class_cd_3']]


jangbi_broken = jangbi_broken.sort_values('job_jisi_dt')

## 기준일자는 오늘로 설정해야 함 > 작업장비이력
jangbi_broken['기준일자'] = '2022-10-11'

jangbi_broken['기준일자'] = pd.to_datetime(jangbi_broken['기준일자'])
jangbi_broken['fac_instdt'] = pd.to_datetime(jangbi_broken['fac_instdt'])
jangbi_broken['job_jisi_dt'] = pd.to_datetime(jangbi_broken['job_jisi_dt'])


jangbi_broken['설치-작업간격'] = jangbi_broken['job_jisi_dt'] - jangbi_broken['fac_instdt']

def slicing(x):
    return x[:-4]
jangbi_broken['설치-작업간격'] = jangbi_broken['설치-작업간격'].astype(str)
jangbi_broken['설치-작업간격'] = jangbi_broken['설치-작업간격'].apply(slicing)
jangbi_broken['설치-작업간격'] = jangbi_broken['설치-작업간격'].astype(int)

jangbi_broken['작업-작업간격'] = 0
# 모두 장애가 발생한 장비이므로 작업횟수 = 1 로 세팅
jangbi_broken['작업횟수'] = 1
jangbi_broken = jangbi_broken.sort_values(['fac_cd','설치-작업간격'])

/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_21116/3023823428.py:4: DtypeWarning: Columns (59) have mixed types. Specify dtype option on import or set low_memory=False.
  FMSA10_작업지시_결과 = pd.read_csv(data+'FMSA10_작업지시_결과_ver2.csv')
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_21116/3023823428.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fmsb03["mat_cd"] = fmsb03["mat_cd"].astype("str")


In [7]:

# 작업-작업 간격 만들기
for i in range(len(jangbi_broken)):
    if i == 0:
        # 젤 처음 오는 경우 > 장비별 가장 오래된 작업
        jangbi_broken['작업-작업간격'].iloc[i] = jangbi_broken['설치-작업간격'].iloc[i]
    else :
        now_data = jangbi_broken['fac_cd'].iloc[i]
        past_data = jangbi_broken['fac_cd'].iloc[i-1]
        # 한 장비가 이전에 장애가 발생했을 경우
        if now_data == past_data:
            jangbi_broken['작업-작업간격'].iloc[i] = jangbi_broken['설치-작업간격'].iloc[i] - jangbi_broken['설치-작업간격'].iloc[i-1]
            jangbi_broken['작업횟수'].iloc[i] = jangbi_broken['작업횟수'].iloc[i-1] +1
        # 장비가 이전에 장애가 발생하지 않았을 경우
        else :
            jangbi_broken['작업-작업간격'].iloc[i] = jangbi_broken['설치-작업간격'].iloc[i]

/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_21116/279619396.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jangbi_broken['작업-작업간격'].iloc[i] = jangbi_broken['설치-작업간격'].iloc[i]
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_21116/279619396.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jangbi_broken['작업-작업간격'].iloc[i] = jangbi_broken['설치-작업간격'].iloc[i] - jangbi_broken['설치-작업간격'].iloc[i-1]
/var/folders/nd/cl2nzwh13zx7jdly2t5m95340000gn/T/ipykernel_21116/279619396.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the d

In [8]:
max_time = datetime.datetime(2022,10,11)

analysis_data = pd.merge(fmsx05,jangbi_broken[['job_jisi_dt','fac_cd','설치-작업간격','작업-작업간격','작업횟수','job_cd','job_nm']], on ='fac_cd', how ='left')

analysis_data['fac_instdt'] = pd.to_datetime(analysis_data['fac_instdt'])

analysis_data = analysis_data[analysis_data['fac_instdt'] <= max_time]


analysis_data['max_time'] = max_time
analysis_data['max_time'] = pd.to_datetime(analysis_data['max_time'])
analysis_data['작업-작업간격'] = analysis_data['작업-작업간격'].fillna(999999)

analysis_data['설치-작업간격'] = analysis_data['설치-작업간격'].fillna(999999)

analysis_data['경과시간'] = 0

def time(df):
    x = df['작업-작업간격']
    if x == 999999:
        return int(str(df['max_time'] -  df['fac_instdt'])[:-13])
    else :
        return int(x)
def time2(df):
    x = df['설치-작업간격']
    if x == 999999:
        return int(str(df['max_time'] -  df['fac_instdt'])[:-13])
    else :
        return int(x)

def time3(df):
    if df['status'] == 0 :
        return int(str(df['max_time'] -  df['fac_instdt'])[:-13])
    else :
        return int(str(df['max_time'] -  df['job_jisi_dt'])[:-13])


def status(df):
    if df['작업횟수'] == 0:
        return 0
    else :
        return 1

analysis_data['작업횟수'] = analysis_data['작업횟수'].fillna(0)
analysis_data['작업-작업간격'] = analysis_data.apply(time,axis=1)
analysis_data['설치-작업간격'] = analysis_data.apply(time2,axis=1)
analysis_data['status'] = analysis_data.apply(status,axis=1)
analysis_data['경과시간'] =  analysis_data.apply(time3,axis=1)
analysis_data = analysis_data.fillna(0)
analysis_data

,bd_cd,bd_nm,fl_cd,fac_cd,fac_nm,jangbi_class_nm_1,jangbi_class_nm_2,jangbi_class_nm_3,jangbi_class_nm_4,fac_instdt,job_jisi_dt,설치-작업간격,작업-작업간격,작업횟수,job_cd,job_nm,max_time,경과시간,status
0,0,0,0,200201040293,(3)세-1F-0(여)로비-화장실) 세면대 2대,기계,"위생, 탱크설비",위생설비,세면대 수전,2018-03-05,0,1681,1681,0.0,0,0,2022-10-11,1681,0
1,0,0,0,200201010257,9F여1대변기(5),기계,"위생, 탱크설비",위생설비,대변기,2014-06-01,0,3054,3054,0.0,0,0,2022-10-11,3054,0
2,0,0,0,200202030001,우수조#1,기계,"위생, 탱크설비",탱크설비,우수 저장조,2019-10-22,0,1085,1085,0.0,0,0,2022-10-11,1085,0
3,0,0,0,200105080599,배기휀#76(EF-504),기계,공기조화설비,환기설비,환기팬,2019-10-23,0,1084,1084,0.0,0,0,2022-10-11,1084,0
4,0,0,0,200202010001,시수조#1,기계,"위생, 탱크설비",탱크설비,상수 저장조,2019-10-22,0,1085,1085,0.0,0,0,2022-10-11,1085,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43584,SJ-SCJS-01,SCC전시동,P006,700101020004,SCC 4호기 권상기,승강기,승객용,MR,권상기,2013-11-12,0,3255,3255,0.0,0,0,2022-10-11,3255,0
43585,SJ-SCJS-01,SCC전시동,P006,700101020005,SCC 5호기 권상기,승강기,승객용,MR,권상기,2013-11-12,0,3255,3255,0.0,0,0,2022-10-11,3255,0
43586,SJ-SCJS-01,SCC전시동,P006,700101020006,SCC 6호기 권상기,승강기,승객용,MR,권상기,2013-11-12,0,3255,3255,0.0,0,0,2022-10-11,3255,0
43587,SJ-SCJS-01,SCC전시동,P006,700201010010,SCC 7호기 제어반,승강기,화물용,로프식,제어반,2014-10-17,0,2916,2916,0.0,0,0,2022-10-11,2916,0


In [10]:
기계 = pd.read_excel(raw_data + '수행자재_완료.xlsx',sheet_name='기계')
통신 = pd.read_excel(raw_data +'수행자재_완료.xlsx',sheet_name='통신')
방재 = pd.read_excel(raw_data +'수행자재_완료.xlsx',sheet_name='방재')
승강기 = pd.read_excel(raw_data +'수행자재_완료.xlsx',sheet_name='승강기')
전기 = pd.read_excel(raw_data +'수행자재_완료.xlsx',sheet_name='전기')

In [11]:
수행범위 = pd.concat([기계,통신,방재,승강기,전기])
input_data= pd.merge(수행범위,analysis_data, how = 'left',left_on=['Lv1','Lv2','Lv3','Lv4'],right_on=['jangbi_class_nm_1','jangbi_class_nm_2','jangbi_class_nm_3','jangbi_class_nm_4'])
input_data.drop(['jangbi_class_nm_1','jangbi_class_nm_2','jangbi_class_nm_3','jangbi_class_nm_4'],axis=1,inplace = True)
input_data['비고'] = input_data['비고'].fillna(0)
input_data = input_data.dropna()

In [12]:
input_data[input_data['그룹정의']=='권상기(승객용)']

,Lv1,Lv2,Lv3,Lv4,장애건수,자재매칭건수,매칭비율,그룹정의,장애분석,자재분석,...,fac_instdt,job_jisi_dt,설치-작업간격,작업-작업간격,작업횟수,job_cd,job_nm,max_time,경과시간,status
32551,승강기,승객용,MR,권상기,0.0,0.0,0.0,권상기(승객용),○,○,...,2013-07-03,0,3387.0,3387.0,0.0,0,0,2022-10-11,3387.0,0.0
32552,승강기,승객용,MR,권상기,0.0,0.0,0.0,권상기(승객용),○,○,...,2013-11-12,0,3255.0,3255.0,0.0,0,0,2022-10-11,3255.0,0.0
32553,승강기,승객용,MR,권상기,0.0,0.0,0.0,권상기(승객용),○,○,...,2013-11-12,0,3255.0,3255.0,0.0,0,0,2022-10-11,3255.0,0.0
32554,승강기,승객용,MR,권상기,0.0,0.0,0.0,권상기(승객용),○,○,...,2013-11-12,0,3255.0,3255.0,0.0,0,0,2022-10-11,3255.0,0.0
32555,승강기,승객용,MR,권상기,0.0,0.0,0.0,권상기(승객용),○,○,...,2014-10-01,0,2932.0,2932.0,0.0,0,0,2022-10-11,2932.0,0.0
32556,승강기,승객용,MR,권상기,0.0,0.0,0.0,권상기(승객용),○,○,...,2014-10-01,0,2932.0,2932.0,0.0,0,0,2022-10-11,2932.0,0.0
32557,승강기,승객용,MR,권상기,0.0,0.0,0.0,권상기(승객용),○,○,...,2012-09-01,0,3692.0,3692.0,0.0,0,0,2022-10-11,3692.0,0.0
32558,승강기,승객용,MR,권상기,0.0,0.0,0.0,권상기(승객용),○,○,...,2014-10-01,0,2932.0,2932.0,0.0,0,0,2022-10-11,2932.0,0.0
32559,승강기,승객용,MR,권상기,0.0,0.0,0.0,권상기(승객용),○,○,...,2014-10-01,0,2932.0,2932.0,0.0,0,0,2022-10-11,2932.0,0.0
32560,승강기,승객용,MR,권상기,0.0,0.0,0.0,권상기(승객용),○,○,...,2014-10-01,0,2932.0,2932.0,0.0,0,0,2022-10-11,2932.0,0.0


In [13]:
수행범위['그룹정의'].unique()

array(['개별냉난방기', '항온항습기', '공조설비/공조기', '공조설비/팬코일', '공조설비/기타', '기타시설',
       '방호, 안전시설', '배관설비', '부속시설/자전거 세척기 및 공기주입기', '부속시설/기타', '열원설비/냉각탑',
       '열원설비/지열히트펌프', '열원설비/냉동기', '열원설비/열교환기', '열원설비/기타', '위생설비/소변기',
       '위생설비/수전', '위생설비/대변기', '위생설비/비데', '위생설비/기타', '자동제어설비', '조경시설',
       '주방시설', '탱크설비', '배수펌프', '냉온수순환 펌프', '오수펌프', '냉각수순환 펌프', '상수 펌프',
       '조경수 펌프', '기타펌프', '우수펌프', '환기설비/배기팬', '환기설비/급기팬', '환기설비/기타',
       '행정망네트워크', nan, '통합SI', '방범망네트워크', 'CCTV', '동작감지', '울타리감지',
       '스피드게이트', '얼굴인식', '출입통제', '통제식회전문', '비상호출', '공용회의실', '전관방송', 'TV',
       '정책홍보', '자동화재탐지설비-감지기', '유도등-피난구 유도등', '소화기구-이동식', '유도등-거실통로 유도등',
       '유도등-복도통로 유도등', '스프링클러설비-밸브', '유도등-계단 유도등', '스프링클러설비-헤드',
       '소방선로-전원', '비상경보설비-비상벨', '자동화재탐지설비-수신기', '방화셔터설비-방화셔터',
       '소화펌프-스프링클러', '소화수조-수조', '소화가스설비-집합관', '소화펌프-겸용', '자동화재탐지설비-중계기',
       '자동화재탐지설비-발신기', '배연창-전원반', '소화전설비-옥내소화전설비', '소화기구-고정식',
       '소화전설비-옥외소화전설비', '제연설비-전실제연설비', '피난기구-완강기', '비상조명등-휴대용 비상조명등',
       '자동화재탐지설비-시각경보기', '소화

In [14]:
input_data.to_csv(data + '생존분석.csv')

- 장비분류(그룹핑), 동, 층, 장비명, 장애발생횟수, 최근장애일자 > 가져와서 돌리면돔